In [1]:
import os.path as osp
import time
import torch
import torch.nn.functional as F
from torch.nn import Linear
from sklearn.metrics import f1_score

from torch_geometric.datasets import PPI
from torch_geometric.loader import DataLoader


path = './ppi_data/'
train_dataset = PPI(path, split='train')
val_dataset = PPI(path, split='val')
test_dataset = PPI(path, split='test')
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

class MLPPPI(torch.nn.Module):
    def __init__(self, input_features, hidden_size, output_classes):
        super().__init__()
        self.fc1 = Linear(input_features, hidden_size)
        self.fc2 = Linear(hidden_size, hidden_size)
        self.fc3 = Linear(hidden_size, output_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Adjust the model initialization to use the new MLP class
input_features = train_dataset.num_features
hidden_size = 256
output_classes = train_dataset.num_classes



# Rest of the code remains the same



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MLPPPI(input_features, hidden_size, output_classes).to(device)
loss_op = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)


def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        loss = loss_op(model(data.x), data.y)  # Only pass data.x
        total_loss += loss.item() * data.num_graphs
        loss.backward()
        optimizer.step()
    return total_loss / len(train_loader.dataset)

@torch.no_grad()
def test(loader):
    model.eval()
    ys, preds = [], []
    for data in loader:
        ys.append(data.y)
        out = model(data.x.to(device))  # Only pass data.x
        preds.append((out > 0).float().cpu())

    y, pred = torch.cat(ys, dim=0).numpy(), torch.cat(preds, dim=0).numpy()
    return f1_score(y, pred, average='micro') if pred.sum() > 0 else 0



times = []
for epoch in range(1, 101):
    start = time.time()
    loss = train()
    val_f1 = test(val_loader)
    test_f1 = test(test_loader)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_f1:.4f}, '
          f'Test: {test_f1:.4f}')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")


Epoch: 001, Loss: 0.5750, Val: 0.4165, Test: 0.4175
Epoch: 002, Loss: 0.5508, Val: 0.4184, Test: 0.4200
Epoch: 003, Loss: 0.5388, Val: 0.4289, Test: 0.4327
Epoch: 004, Loss: 0.5307, Val: 0.4634, Test: 0.4685
Epoch: 005, Loss: 0.5197, Val: 0.4916, Test: 0.4987
Epoch: 006, Loss: 0.5121, Val: 0.5195, Test: 0.5270
Epoch: 007, Loss: 0.5100, Val: 0.5034, Test: 0.5106
Epoch: 008, Loss: 0.5057, Val: 0.4946, Test: 0.5026
Epoch: 009, Loss: 0.4986, Val: 0.5027, Test: 0.5108
Epoch: 010, Loss: 0.4949, Val: 0.4954, Test: 0.5040
Epoch: 011, Loss: 0.4916, Val: 0.5034, Test: 0.5125
Epoch: 012, Loss: 0.4872, Val: 0.5221, Test: 0.5310
Epoch: 013, Loss: 0.4854, Val: 0.5060, Test: 0.5161
Epoch: 014, Loss: 0.4822, Val: 0.5021, Test: 0.5114
Epoch: 015, Loss: 0.4815, Val: 0.5205, Test: 0.5297
Epoch: 016, Loss: 0.4824, Val: 0.5397, Test: 0.5491
Epoch: 017, Loss: 0.4768, Val: 0.5440, Test: 0.5535
Epoch: 018, Loss: 0.4769, Val: 0.5309, Test: 0.5403
Epoch: 019, Loss: 0.4747, Val: 0.5298, Test: 0.5392
Epoch: 020, 